In [156]:
import numpy as np

import matplotlib as mpl
import matplotlib
# matplotlib.use('agg')
# matplotlib.use('TkAgg')
# matplotlib.use('qtagg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import *
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                            AutoMinorLocator)
from math import *
#import pandas as pd

from scipy import interpolate
from scipy.optimize import curve_fit 

from astropy.io import fits
from astropy.table import Table
from astropy.table import vstack
from astropy.coordinates import SkyCoord, ICRS, Galactic
# import astropy.units as u
import astropy.units as units
import astropy.coordinates as coord
#from matplotlib.colors import LogNorm

# from mpl_toolkits.axes_grid1 import make_axes_locatable
# #import matplotlib.ticker as mtick


import os
#%matplotlib widget
import seaborn as sns
import os, sys


# from astropy.io import ascii
# from astropy.coordinates import galactocentric_frame_defaults

import sklearn
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

from scipy import interpolate

import random
from scipy import integrate


gcolor = ['c','blue','g','r','orange', 'green','cyan']

In [157]:
import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
import ppxf.sps_util as lib
import sys,glob
from pathlib import Path

from ppxf.ppxf import ppxf, robust_sigma

from vorbin.voronoi_2d_binning import voronoi_2d_binning
from plotbin.display_bins import display_bins

from tqdm import tqdm  

In [158]:
def TB_reindex(TB_now):
    TB_now = TB_now.reset_index()
    TB_now = TB_now.drop(columns='index')
    return TB_now

In [159]:
def Data_Grouping(array_x, array_y, Group_K):
    Len = len(array_x)
    GP_num = int(Len/Group_K)
    if(Len%Group_K>0):
        GP_num += 1
    rt_x = []
    sig_x = []
    rt_y = []
    sig_y = []
    for i in range(GP_num):
        sum_x = []
        sum_y = []
        for t in range(Group_K):
            if((i*Group_K + t)< Len):
                sum_x += [array_x[i*Group_K + t]]
                sum_y += [array_y[i*Group_K + t]]
        rt_x += [np.median(sum_x)]
        sig_x += [np.std(sum_x)]
        rt_y += [np.median(sum_y)]
        sig_y += [np.std(sum_y)]

    return rt_x, rt_y, sig_x, sig_y

In [160]:
c = 299792.458  # speed of light [km/s]

galaxy_name = 'VCC_1588'
spectrum_filename = 'VCC1588_stack.fits'
spectrum_z = 0.0042

In [161]:
import re
import csv
import os

def process_and_merge_data(header1, data1, header2, data2, output_path):
    def standardize_minus(s):
        return s.replace('−', '-')
    
    def process_header(h):
        h = standardize_minus(h)
        # 先进行所有的替换
        # header_mapping = {
        #     '[Z/H]': 'ZoH',
        #     'Mg b': 'Mgb',
        #     'H ': 'Hb ',
        # }
        # for old, new in header_mapping.items():
        #     h = h.replace(old, new)
        
        # 分割并清理空格
        headers = [x.strip() for x in h.split() if x.strip()]
        
        # 确保Age和ZoH在正确的位置
        if 'Age' not in headers:
            headers.insert(0, 'Age')
        if 'ZoH' not in headers:
            headers.insert(1, 'ZoH')
        # if 'alpha_Fe' not in headers:
        #     headers.insert(2, 'alpha_Fe')
            
        return headers
    
    def process_data_line(line):
        line = standardize_minus(line)
        return [x.strip() for x in line.split() if x.strip()]
    
    # 处理两组数据
    headers1 = process_header(header1)
    headers2 = process_header(header2)
    
    # 移除headers2中重复的Age, ZoH和alpha_Fe列
    unique_headers2 = [h for h in headers2 if h not in ['Age', 'ZoH']]
    final_headers = headers1 + unique_headers2

    # 处理数据行
    data_lines1 = [process_data_line(line) for line in data1.strip().split('\n')]
    data_lines2 = [process_data_line(line) for line in data2.strip().split('\n')]
    
    # 创建数据字典以便合并
    data_dict = {}
    for line in data_lines1:
        key = (line[0], line[1])  # (Age, ZoH)
        data_dict[key] = line
    
    # 合并数据
    merged_data = []
    for line in data_lines2:
        key = (line[0], line[1])  # (Age, ZoH)
        if key in data_dict:
            # 去除第二组数据中的Age, ZoH, alpha_Fe列
            merged_line = data_dict[key] + line[2:]
            merged_data.append(merged_line)
    
    # 确保输出目录存在
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 保存为CSV文件
    try:
        with open(output_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(final_headers)
            writer.writerows(merged_data)
        print(f"合并后的数据已成功保存到: {output_path}")
        # 打印一些信息来帮助验证
        print(f"表头数量: {len(final_headers)}")
        print(f"数据行数: {len(merged_data)}")
        print(f"每行数据列数: {len(merged_data[0]) if merged_data else 0}")
    except Exception as e:
        print(f"保存文件时出错: {str(e)}")
    
    return final_headers, merged_data

# 使用示例
header1 = "Age ZoH CN1 CN2 Ca4227 G4300 Fe4383 Ca4455 Fe4531 C24668 Hb Fe5015"
header2 = "Age ZoH Mg1 Mg2 Mgb Fe5270 Fe5335 Fe5406 Fe5709 Fe5782 NaD TiO1 TiO2"

In [162]:
D_1_1 = '''
1 −2.25 −0.2632 −0.1536 −0.4260 −4.3872 −2.8764 −0.1924 −0.8226 1.6082 6.7915 −0.0725
2 −2.25 −0.2033 −0.1214 −0.2038 −2.4775 −1.8406 −0.0604 −0.3240 0.9274 4.8015 0.3634
3 −2.25 −0.1871 −0.1137 −0.1132 −1.9934 −1.4889 −0.0243 −0.1584 0.6396 4.3819 0.4838
4 −2.25 −0.1740 −0.1077 −0.0440 −1.6085 −1.2146 −0.0036 −0.0539 0.4111 4.0630 0.5427
5 −2.25 −0.1621 −0.1021 0.0173 −1.2265 −0.9724 0.0149 0.0312 0.2273 3.7640 0.5811
6 −2.25 −0.1491 −0.0941 0.0661 −0.8894 −0.6061 0.0511 0.1253 0.1278 3.5048 0.6476
8 −2.25 −0.1324 −0.0868 0.1459 −0.4385 −0.2853 0.0596 0.1951 −0.1079 3.0582 0.6530
10 −2.25 −0.1251 −0.0826 0.1764 −0.3607 0.0315 0.0778 0.2360 −0.1940 2.9791 0.6651
12 −2.25 −0.1240 −0.0821 0.1694 −0.5518 0.1609 0.0791 0.2067 −0.2386 3.0454 0.6450
15 −2.25 −0.1229 −0.0825 0.2547 −0.0944 0.2559 0.1224 0.2624 −0.2508 2.6915 0.6520
1 −1.35 −0.1830 −0.1023 −0.0251 −1.4389 −0.6485 0.0989 0.6231 0.0868 4.7632 1.3209
2 −1.35 −0.1677 −0.0952 0.0846 −0.9344 −0.4518 0.1603 0.8179 −0.0869 4.3492 1.4799
3 −1.35 −0.1507 −0.0858 0.1756 −0.3594 −0.1221 0.2292 0.9923 −0.1338 3.9819 1.6341
4 −1.35 −0.1330 −0.0753 0.2539 0.2261 0.2575 0.2983 1.1480 −0.1285 3.5911 1.7713
5 −1.35 −0.1170 −0.0654 0.3213 0.7783 0.5915 0.3589 1.2793 −0.1167 3.2040 1.8973
6 −1.35 −0.1054 −0.0580 0.3712 1.1612 0.8880 0.4074 1.3776 −0.0940 2.9284 1.9845
8 −1.35 −0.0914 −0.0497 0.4435 1.6581 1.1964 0.4593 1.4837 −0.1069 2.5635 2.0734
10 −1.35 −0.0820 −0.0437 0.4986 2.0032 1.4411 0.5041 1.5701 −0.1004 2.3276 2.1531
12 −1.35 −0.0792 −0.0416 0.5146 2.0610 1.5533 0.5243 1.6045 −0.0941 2.2644 2.1744
15 −1.35 −0.1055 −0.0576 0.4669 1.3267 1.1610 0.4601 1.4697 −0.1794 2.6002 2.0480
1 −0.33 −0.1095 −0.0515 0.4147 1.1773 1.6031 0.7138 2.0232 1.4378 3.8672 3.4603
2 −0.33 −0.0655 −0.0204 0.6320 2.6899 2.6139 0.9529 2.4402 2.1475 2.9290 3.9201
3 −0.33 −0.0501 −0.0095 0.7272 3.2282 3.0163 1.0370 2.5855 2.4061 2.5892 4.0585
4 −0.33 −0.0392 −0.0017 0.7914 3.6089 3.3134 1.0959 2.6813 2.5487 2.3659 4.1519
5 −0.33 −0.0328 0.0025 0.8330 3.8434 3.5038 1.1314 2.7374 2.6158 2.2338 4.2088
6 −0.33 −0.0286 0.0052 0.8707 4.0287 3.6716 1.1614 2.7840 2.6643 2.1407 4.2429
8 −0.33 −0.0213 0.0102 0.9362 4.3259 3.9266 1.2165 2.8689 2.7726 1.9821 4.3204
10 −0.33 −0.0138 0.0162 1.0030 4.6071 4.1882 1.2807 2.9645 2.9265 1.8302 4.4491
12 −0.33 −0.0079 0.0209 1.0495 4.8227 4.3964 1.3380 3.0436 3.0289 1.7111 4.5827
15 −0.33 −0.0059 0.0233 1.0946 4.9454 4.6012 1.3920 3.1201 3.1204 1.6192 4.6370
1 0.00 −0.0921 −0.0381 0.5382 1.6441 1.9495 0.9164 2.2642 2.5692 4.0452 3.9143
2 0.00 −0.0242 0.0145 0.8315 3.7492 3.5803 1.2859 2.8785 3.9158 2.6023 4.6825
3 0.00 0.0046 0.0399 0.9531 4.4635 4.3642 1.4378 3.1023 4.5470 2.2370 5.0994
4 0.00 0.0129 0.0464 1.0193 4.7444 4.6323 1.4920 3.1812 4.7126 2.1088 5.1972
5 0.00 0.0222 0.0546 1.0802 4.9975 4.8914 1.5474 3.2631 4.8916 1.9859 5.2863
6 0.00 0.0294 0.0609 1.1259 5.2014 5.0871 1.5912 3.3244 5.0260 1.8919 5.3676
8 0.00 0.0327 0.0629 1.1755 5.3741 5.2233 1.6262 3.3641 5.0860 1.8085 5.4434
10 0.00 0.0396 0.0692 1.2394 5.5640 5.4402 1.6788 3.4376 5.2252 1.6981 5.5171
12 0.00 0.0491 0.0786 1.3043 5.7548 5.7077 1.7423 3.5326 5.4185 1.5771 5.5834
15 0.00 0.0604 0.0902 1.3825 5.9363 5.9742 1.8202 3.6367 5.6310 1.4468 5.7254
1 0.35 −0.0275 0.0174 0.8320 3.1800 3.5344 1.4137 2.9902 5.2854 3.1441 5.2526
2 0.35 0.0351 0.0720 1.1133 4.6679 5.3528 1.7150 3.4519 6.5652 2.2466 5.8858
3 0.35 0.0450 0.0793 1.2116 5.0609 5.7139 1.7865 3.5515 6.7700 2.0765 6.0178
4 0.35 0.0575 0.0906 1.3060 5.3484 6.0373 1.8600 3.6629 7.0214 1.9218 6.1383
5 0.35 0.0642 0.0966 1.3747 5.5507 6.2647 1.9114 3.7403 7.1833 1.8150 6.2045
6 0.35 0.0713 0.1033 1.4326 5.7039 6.4627 1.9568 3.8095 7.3349 1.7257 6.2601
8 0.35 0.0785 0.1100 1.5128 5.8589 6.6713 2.0075 3.8826 7.4653 1.6198 6.3117
10 0.35 0.0883 0.1197 1.5854 5.9921 6.8803 2.0606 3.9572 7.6318 1.5211 6.3817
12 0.35 0.1022 0.1345 1.6810 6.0991 7.1384 2.1297 4.0616 7.8734 1.4119 6.4755
15 0.35 0.1211 0.1548 1.7990 6.1712 7.4315 2.2125 4.1897 8.1654 1.2864 6.5747
1 0.67 −0.0297 0.0095 0.9266 3.6767 3.9129 1.6121 3.1503 6.3317 3.0753 5.6368
2 0.67 0.0338 0.0654 1.2963 5.2506 6.3496 1.9696 3.7168 7.9710 2.2034 6.3998
3 0.67 0.0797 0.1123 1.5822 5.7968 7.2090 2.1878 4.0752 9.0789 1.8009 6.8336
4 0.67 0.1093 0.1437 1.7612 6.0429 7.7054 2.3126 4.2802 9.6626 1.5869 7.0322
5 0.67 0.1184 0.1534 1.7866 6.1082 7.8680 2.3535 4.3478 9.8143 1.5168 7.0785
6 0.67 0.1265 0.1624 1.8305 6.1658 7.9832 2.3844 4.4006 9.9484 1.4644 7.1185
8 0.67 0.1458 0.1836 1.9554 6.2496 8.2480 2.4552 4.5205 10.2522 1.3464 7.2093
10 0.67 0.1718 0.2123 2.1660 6.2938 8.5831 2.5448 4.6691 10.6289 1.2017 7.3223
12 0.67 0.2022 0.2460 2.2959 6.2173 8.7759 2.6024 4.7804 10.9291 1.1196 7.4096
15 0.67 0.2626 0.3117 2.4246 6.0272 9.0842 2.6630 4.9298 11.4026 1.0227 7.3986
'''

In [163]:
D_1_2 = '''
1 −2.25 0.0115 −0.0025 0.1794 0.1715 0.2673 −0.1296 −0.0573 −0.4248 0.8827 0.0140 −0.0048
2 −2.25 0.0023 0.0094 0.3366 0.3408 0.4326 0.0056 −0.0074 −0.3342 0.8046 0.0132 −0.0037
3 −2.25 0.0012 0.0123 0.3632 0.4333 0.4781 0.0519 0.0094 −0.3070 0.8133 0.0125 −0.0039
4 −2.25 0.0003 0.0167 0.4429 0.4910 0.5023 0.0725 0.0119 −0.2974 0.8275 0.0121 −0.0042
5 −2.25 0.0000 0.0205 0.5125 0.5421 0.5246 0.0835 0.0140 −0.2900 0.8497 0.0117 −0.0045
6 −2.25 −0.0011 0.0230 0.5364 0.6168 0.5902 0.1264 0.0248 −0.2677 0.8524 0.0114 −0.0047
8 −2.25 0.0007 0.0318 0.7516 0.6302 0.5827 0.1394 0.0184 −0.2772 0.9264 0.0108 −0.0052
10 −2.25 0.0019 0.0340 0.7594 0.6900 0.6335 0.1781 0.0242 −0.2646 0.9746 0.0103 −0.0057
12 −2.25 0.0047 0.0354 0.7752 0.7227 0.6713 0.2069 0.0317 −0.2587 1.0268 0.0099 −0.0062
15 −2.25 0.0102 0.0423 0.8700 0.7250 0.7312 0.2321 0.0371 −0.2557 1.1413 0.0098 −0.0054
1 −1.35 0.0105 0.0322 0.6423 0.3784 0.4942 0.2456 0.2880 0.0919 0.8282 0.0138 0.0006
2 −1.35 0.0085 0.0392 0.7608 0.4677 0.5650 0.2894 0.2997 0.1121 0.8677 0.0139 0.0015
3 −1.35 0.0087 0.0450 0.8613 0.5814 0.6334 0.3350 0.3167 0.1307 0.9145 0.0140 0.0023
4 −1.35 0.0095 0.0509 0.9603 0.6977 0.7174 0.3835 0.3335 0.1520 0.9684 0.0141 0.0031
5 −1.35 0.0108 0.0563 1.0447 0.7946 0.7838 0.4229 0.3514 0.1675 1.0109 0.0143 0.0038
6 −1.35 0.0120 0.0607 1.0997 0.8776 0.8532 0.4646 0.3646 0.1837 1.0572 0.0145 0.0044
8 −1.35 0.0148 0.0677 1.2042 0.9597 0.9125 0.5108 0.3770 0.1932 1.1322 0.0146 0.0051
10 −1.35 0.0174 0.0730 1.2624 1.0333 0.9713 0.5565 0.3909 0.2020 1.1978 0.0149 0.0059
12 −1.35 0.0194 0.0768 1.2523 1.0773 1.0199 0.5900 0.3932 0.2123 1.2605 0.0151 0.0064
15 −1.35 0.0231 0.0747 1.1922 0.9890 0.9936 0.5823 0.3846 0.1990 1.3362 0.0152 0.0065
1 −0.33 0.0315 0.1029 1.5168 1.6812 1.3849 0.9656 0.7457 0.5721 1.5458 0.0215 0.0217
2 −0.33 0.0407 0.1262 1.8916 1.9694 1.6762 1.1566 0.7870 0.6250 1.8194 0.0282 0.0371
3 −0.33 0.0448 0.1363 2.0499 2.0735 1.7740 1.2066 0.7917 0.6380 1.9504 0.0316 0.0445
4 −0.33 0.0479 0.1430 2.1492 2.1472 1.8426 1.2471 0.8001 0.6469 2.0343 0.0327 0.0480
5 −0.33 0.0498 0.1473 2.2200 2.1927 1.8825 1.2701 0.8042 0.6507 2.0932 0.0332 0.0497
6 −0.33 0.0519 0.1518 2.2820 2.2298 1.9267 1.3048 0.8064 0.6523 2.1537 0.0338 0.0521
8 −0.33 0.0562 0.1612 2.4310 2.3025 1.9942 1.3460 0.8133 0.6603 2.2673 0.0351 0.0562
10 −0.33 0.0612 0.1713 2.5780 2.3913 2.0714 1.3944 0.8245 0.6726 2.3855 0.0370 0.0611
12 −0.33 0.0659 0.1797 2.6874 2.4710 2.1466 1.4542 0.8384 0.6852 2.4852 0.0382 0.0652
15 −0.33 0.0735 0.1891 2.7645 2.5386 2.2154 1.5069 0.8554 0.6969 2.6203 0.0394 0.0700
1 0.00 0.0386 0.1265 1.9003 2.0163 1.7421 1.0617 0.8153 0.6344 1.9756 0.0212 0.0212
2 0.00 0.0621 0.1694 2.5229 2.4891 2.2064 1.3886 0.9118 0.7398 2.4554 0.0329 0.0480
3 0.00 0.0707 0.1912 2.8583 2.6974 2.3530 1.5248 0.9572 0.7877 2.6897 0.0389 0.0616
4 0.00 0.0751 0.2009 3.0203 2.7613 2.4185 1.5653 0.9666 0.7973 2.7755 0.0400 0.0653
5 0.00 0.0798 0.2105 3.1758 2.8288 2.4854 1.6227 0.9810 0.8117 2.8678 0.0407 0.0690
6 0.00 0.0832 0.2174 3.2906 2.8815 2.5363 1.6557 0.9926 0.8222 2.9333 0.0411 0.0710
8 0.00 0.0856 0.2238 3.4315 2.9208 2.5736 1.6724 0.9932 0.8206 3.0036 0.0416 0.0736
10 0.00 0.0908 0.2338 3.5816 2.9839 2.6361 1.7148 1.0010 0.8292 3.1227 0.0426 0.0786
12 0.00 0.0980 0.2456 3.7095 3.0601 2.7126 1.7742 1.0132 0.8443 3.2697 0.0441 0.0853
15 0.00 0.1059 0.2588 3.8687 3.1587 2.8050 1.8349 1.0239 0.8600 3.4501 0.0464 0.0936
1 0.35 0.0692 0.1829 2.5531 2.7286 2.5285 1.5986 1.0368 0.8610 3.0344 0.0357 0.0546
2 0.35 0.0926 0.2312 3.3280 3.1022 2.9100 1.8600 1.1289 0.9470 3.4668 0.0407 0.0691
3 0.35 0.0980 0.2429 3.5572 3.1745 2.9835 1.9153 1.1469 0.9578 3.5576 0.0411 0.0716
4 0.35 0.1056 0.2561 3.7602 3.2643 3.0764 1.9805 1.1609 0.9717 3.7039 0.0424 0.0765
5 0.35 0.1111 0.2654 3.8922 3.3217 3.1383 2.0219 1.1728 0.9824 3.8103 0.0431 0.0806
6 0.35 0.1163 0.2741 4.0068 3.3739 3.1923 2.0631 1.1830 0.9934 3.9135 0.0440 0.0848
8 0.35 0.1217 0.2841 4.1581 3.4317 3.2485 2.1013 1.1874 0.9977 4.0443 0.0446 0.0895
10 0.35 0.1268 0.2936 4.2969 3.4947 3.3067 2.1416 1.1900 1.0030 4.1796 0.0456 0.0947
12 0.35 0.1350 0.3067 4.4400 3.5842 3.3898 2.2023 1.1971 1.0196 4.3716 0.0475 0.1025
15 0.35 0.1455 0.3228 4.5896 3.6941 3.4895 2.2750 1.2057 1.0446 4.6329 0.0499 0.1134
1 0.67 0.0727 0.1826 2.5303 2.8486 2.8966 1.7422 1.1642 0.8932 3.3312 0.0255 0.0348
2 0.67 0.1064 0.2555 3.6701 3.3038 3.3763 2.0992 1.2638 0.9982 4.0502 0.0392 0.0685
3 0.67 0.1345 0.3009 4.2974 3.6167 3.6995 2.3412 1.3354 1.0935 4.6377 0.0461 0.0867
4 0.67 0.1520 0.3284 4.6380 3.7877 3.8600 2.4736 1.3663 1.1393 4.9842 0.0490 0.0955
5 0.67 0.1574 0.3370 4.7429 3.8423 3.8996 2.5119 1.3725 1.1480 5.0196 0.0490 0.0960
6 0.67 0.1620 0.3443 4.8261 3.8896 3.9442 2.5436 1.3800 1.1597 5.1099 0.0489 0.0965
8 0.67 0.1724 0.3608 5.0166 3.9950 4.0435 2.6140 1.3965 1.1855 5.3703 0.0496 0.0999
10 0.67 0.1852 0.3811 5.2568 4.1221 4.1634 2.6987 1.4160 1.2161 5.7720 0.0517 0.1074
12 0.67 0.1951 0.3966 5.4092 4.2341 4.2649 2.7597 1.4223 1.2432 6.0471 0.0529 0.1111
15 0.67 0.2082 0.4157 5.5238 4.3882 4.3884 2.8348 1.4386 1.2902 6.4622 0.0527 0.1147
'''

In [164]:
D_2_1 = '''
1 −2.25 −0.2597 −0.1509 −0.4300 −4.3330 −3.0443 −0.1991 −0.8268 1.5056 6.9087 −0.0714
2 −2.25 −0.2023 −0.1210 −0.2076 −2.4231 −1.9632 −0.0649 −0.3439 0.8304 4.9274 0.3419
3 −2.25 −0.1864 −0.1137 −0.1142 −1.9177 −1.6542 −0.0261 −0.1779 0.5513 4.5009 0.4577
4 −2.25 −0.1734 −0.1078 −0.0414 −1.5309 −1.3888 −0.0044 −0.0702 0.3322 4.1729 0.5150
5 −2.25 −0.1614 −0.1022 0.0234 −1.1476 −1.1470 0.0150 0.0177 0.1563 3.8651 0.5525
6 −2.25 −0.1483 −0.0942 0.0755 −0.8130 −0.8060 0.0526 0.1132 0.0644 3.5966 0.6159
8 −2.25 −0.1312 −0.0866 0.1525 −0.3758 −0.4597 0.0592 0.1846 −0.1619 3.1330 0.6225
10 −2.25 −0.1235 −0.0820 0.1819 −0.3228 −0.1394 0.0765 0.2219 −0.2400 3.0434 0.6345
12 −2.25 −0.1218 −0.0811 0.1743 −0.5484 0.0154 0.0767 0.1929 −0.2791 3.1037 0.6161
15 −2.25 −0.1199 −0.0808 0.2492 −0.0946 0.1810 0.1195 0.2443 −0.2847 2.7412 0.6237
1 −1.35 −0.1786 −0.0983 −0.0399 −1.2897 −1.1034 0.1033 0.5457 0.0440 4.8816 1.2372
2 −1.35 −0.1654 −0.0932 0.0702 −0.8019 −0.7954 0.1646 0.7393 −0.1358 4.4698 1.3944
3 −1.35 −0.1489 −0.0842 0.1627 −0.2178 −0.4851 0.2350 0.9112 −0.1833 4.0928 1.5444
4 −1.35 −0.1312 −0.0736 0.2430 0.3744 −0.1263 0.3047 1.0646 −0.1769 3.6896 1.6767
5 −1.35 −0.1150 −0.0636 0.3127 0.9343 0.1945 0.3660 1.1937 −0.1636 3.2898 1.7972
6 −1.35 −0.1034 −0.0562 0.3648 1.3206 0.4731 0.4154 1.2904 −0.1380 3.0048 1.8802
8 −1.35 −0.0888 −0.0474 0.4358 1.8199 0.7827 0.4663 1.3935 −0.1499 2.6274 1.9650
10 −1.35 −0.0788 −0.0409 0.4908 2.1664 1.0271 0.5104 1.4734 −0.1396 2.3819 2.0399
12 −1.35 −0.0753 −0.0384 0.5073 2.2127 1.1558 0.5301 1.5049 −0.1295 2.3124 2.0593
15 −1.35 −0.1020 −0.0547 0.4598 1.4096 0.8981 0.4627 1.3759 −0.2111 2.6446 1.9398
1 −0.33 −0.1047 −0.0464 0.3875 1.3806 0.9236 0.7220 1.8809 1.3543 3.9736 3.2775
2 −0.33 −0.0608 −0.0154 0.6048 2.9210 1.8994 0.9623 2.2841 2.0340 3.0051 3.7200
3 −0.33 −0.0456 −0.0046 0.6999 3.4671 2.2681 1.0459 2.4227 2.2804 2.6552 3.8552
4 −0.33 −0.0345 0.0032 0.7645 3.8547 2.5397 1.1043 2.5140 2.4171 2.4251 3.9456
5 −0.33 −0.0281 0.0076 0.8067 4.0929 2.7162 1.1391 2.5671 2.4809 2.2891 4.0012
6 −0.33 −0.0239 0.0102 0.8449 4.2812 2.8702 1.1684 2.6106 2.5274 2.1935 4.0351
8 −0.33 −0.0162 0.0156 0.9101 4.5858 3.0984 1.2221 2.6892 2.6298 2.0304 4.1094
10 −0.33 −0.0080 0.0222 0.9780 4.8785 3.3230 1.2862 2.7772 2.7787 1.8736 4.2294
12 −0.33 −0.0013 0.0276 1.0270 5.1069 3.4937 1.3443 2.8502 2.8783 1.7499 4.3519
15 −0.33 0.0019 0.0310 1.0763 5.2407 3.6526 1.3994 2.9206 2.9652 1.6533 4.3985
1 0.00 −0.0888 −0.0343 0.5022 1.8328 1.2898 0.9142 2.1059 2.4344 4.1623 3.7398
2 0.00 −0.0192 0.0201 0.7947 3.9855 2.7794 1.2869 2.6962 3.7332 2.6718 4.4719
3 0.00 0.0113 0.0474 0.9196 4.7289 3.4646 1.4455 2.9126 4.3490 2.2916 4.8547
4 0.00 0.0199 0.0542 0.9856 5.0160 3.7057 1.4988 2.9864 4.5070 2.1599 4.9491
5 0.00 0.0296 0.0629 1.0468 5.2776 3.9303 1.5542 3.0631 4.6792 2.0332 5.0331
6 0.00 0.0373 0.0696 1.0926 5.4892 4.0986 1.5977 3.1200 4.8078 1.9364 5.1100
8 0.00 0.0407 0.0717 1.1418 5.6678 4.2158 1.6300 3.1536 4.8609 1.8512 5.1841
10 0.00 0.0481 0.0785 1.2063 5.8693 4.3891 1.6819 3.2203 4.9922 1.7376 5.2533
12 0.00 0.0586 0.0889 1.2740 6.0766 4.5898 1.7463 3.3078 5.1769 1.6130 5.3125
15 0.00 0.0713 0.1020 1.3572 6.2838 4.7632 1.8258 3.4029 5.3791 1.4785 5.4399
1 0.35 −0.0234 0.0225 0.7918 3.3950 2.7150 1.4115 2.8004 5.0545 3.2313 5.0212
2 0.35 0.0426 0.0807 1.0749 4.9264 4.3325 1.7191 3.2422 6.2961 2.3032 5.6146
3 0.35 0.0531 0.0884 1.1724 5.3252 4.6662 1.7872 3.3338 6.4897 2.1288 5.7456
4 0.35 0.0664 0.1006 1.2670 5.6223 4.9414 1.8602 3.4372 6.7306 1.9695 5.8594
5 0.35 0.0736 0.1071 1.3358 5.8320 5.1308 1.9103 3.5075 6.8842 1.8598 5.9225
6 0.35 0.0813 0.1146 1.3942 5.9935 5.2868 1.9550 3.5700 7.0272 1.7681 5.9742
8 0.35 0.0890 0.1217 1.4757 6.1588 5.4444 2.0034 3.6333 7.1469 1.6599 6.0242
10 0.35 0.0996 0.1323 1.5521 6.3065 5.5838 2.0557 3.6991 7.3008 1.5587 6.0878
12 0.35 0.1147 0.1483 1.6525 6.4300 5.7550 2.1257 3.7943 7.5289 1.4463 6.1716
15 0.35 0.1352 0.1705 1.7795 6.5299 5.9058 2.2109 3.9104 7.8073 1.3170 6.2554
1 0.67 −0.0279 0.0120 0.8774 3.8711 3.1506 1.5873 2.9447 6.0333 3.1680 5.4444
2 0.67 0.0383 0.0708 1.2455 5.4788 5.3464 1.9458 3.4809 7.6192 2.2673 6.1689
3 0.67 0.0877 0.1214 1.5350 6.0524 6.0315 2.1705 3.8179 8.6972 1.8509 6.5627
4 0.67 0.1202 0.1560 1.7183 6.3190 6.3973 2.3004 4.0084 9.2641 1.6296 6.7370
5 0.67 0.1298 0.1662 1.7436 6.3882 6.5236 2.3394 4.0677 9.4061 1.5577 6.7814
6 0.67 0.1390 0.1763 1.7869 6.4516 6.5898 2.3696 4.1129 9.5309 1.5039 6.8153
8 0.67 0.1601 0.1995 1.9171 6.5527 6.7402 2.4416 4.2178 9.8190 1.3828 6.8925
10 0.67 0.1887 0.2309 2.1398 6.6229 6.9323 2.5329 4.3510 10.1741 1.2345 6.9886
12 0.67 0.2209 0.2667 2.2779 6.5607 7.0150 2.5933 4.4517 10.4574 1.1500 7.0598
15 0.67 0.2848 0.3365 2.4229 6.3954 7.1129 2.6584 4.5845 10.9122 1.0507 7.0285
'''

In [165]:
D_2_2 = '''
1 −2.25 0.0141 −0.0026 0.2098 0.1039 0.2060 −0.1542 −0.0526 −0.4449 0.8640 0.0137 −0.0053
2 −2.25 0.0049 0.0101 0.3873 0.2550 0.3434 −0.0306 −0.0067 −0.3484 0.7862 0.0129 −0.0041
3 −2.25 0.0035 0.0131 0.4097 0.3476 0.3878 0.0146 0.0087 −0.3190 0.8009 0.0122 −0.0043
4 −2.25 0.0023 0.0177 0.4998 0.4090 0.4065 0.0395 0.0110 −0.3081 0.8205 0.0118 −0.0046
5 −2.25 0.0020 0.0217 0.5789 0.4634 0.4238 0.0566 0.0128 −0.2998 0.8473 0.0114 −0.0049
6 −2.25 0.0009 0.0244 0.6040 0.5357 0.4754 0.0969 0.0226 −0.2749 0.8532 0.0111 −0.0051
8 −2.25 0.0028 0.0338 0.8624 0.5479 0.4686 0.1047 0.0165 −0.2854 0.9347 0.0105 −0.0057
10 −2.25 0.0043 0.0361 0.8696 0.5998 0.5080 0.1342 0.0215 −0.2716 0.9877 0.0101 −0.0062
12 −2.25 0.0072 0.0376 0.8868 0.6280 0.5373 0.1560 0.0282 −0.2647 1.0434 0.0096 −0.0067
15 −2.25 0.0123 0.0446 0.9783 0.6327 0.5879 0.1739 0.0327 −0.2628 1.1445 0.0095 −0.0069
1 −1.35 0.0134 0.0349 0.7672 0.2644 0.3664 0.1511 0.2610 0.0186 0.8196 0.0137 0.0001
2 −1.35 0.0121 0.0422 0.8957 0.3523 0.4343 0.1989 0.2707 0.0466 0.8577 0.0136 0.0009
3 −1.35 0.0123 0.0483 1.0058 0.4662 0.5018 0.2434 0.2852 0.0666 0.9063 0.0137 0.0017
4 −1.35 0.0134 0.0546 1.1173 0.5814 0.5698 0.2908 0.2995 0.0871 0.9630 0.0139 0.0024
5 −1.35 0.0149 0.0604 1.2139 0.6779 0.6233 0.3305 0.3150 0.1022 1.0084 0.0141 0.0031
6 −1.35 0.0165 0.0651 1.2766 0.7589 0.6786 0.3692 0.3264 0.1199 1.0573 0.0143 0.0037
8 −1.35 0.0196 0.0725 1.3990 0.8311 0.7257 0.4030 0.3366 0.1271 1.1363 0.0145 0.0043
10 −1.35 0.0227 0.0782 1.4686 0.8952 0.7717 0.4371 0.3485 0.1368 1.2062 0.0148 0.0052
12 −1.35 0.0247 0.0824 1.4570 0.9333 0.8096 0.4622 0.3501 0.1472 1.2728 0.0151 0.0056
15 −1.35 0.0280 0.0802 1.3923 0.8561 0.7879 0.4532 0.3413 0.1353 1.3496 0.0152 0.0055
1 −0.33 0.0393 0.1111 1.8100 1.4380 1.0876 0.7377 0.6705 0.4497 1.5597 0.0216 0.0206
2 −0.33 0.0501 0.1361 2.2423 1.7055 1.3360 0.9008 0.7049 0.5013 1.8342 0.0286 0.0357
3 −0.33 0.0552 0.1468 2.4193 1.8034 1.4224 0.9433 0.7070 0.5144 1.9644 0.0322 0.0429
4 −0.33 0.0589 0.1539 2.5309 1.8727 1.4778 0.9782 0.7132 0.5232 2.0483 0.0333 0.0463
5 −0.33 0.0611 0.1585 2.6089 1.9161 1.5100 0.9986 0.7157 0.5271 2.1069 0.0337 0.0479
6 −0.33 0.0636 0.1631 2.6771 1.9510 1.5454 1.0269 0.7165 0.5299 2.1673 0.0344 0.0503
8 −0.33 0.0685 0.1731 2.8425 2.0134 1.5986 1.0559 0.7210 0.5367 2.2827 0.0357 0.0544
10 −0.33 0.0743 0.1839 3.0120 2.0889 1.6587 1.0915 0.7301 0.5484 2.4050 0.0377 0.0591
12 −0.33 0.0797 0.1929 3.1421 2.1558 1.7161 1.1375 0.7420 0.5602 2.5100 0.0390 0.0632
15 −0.33 0.0886 0.2031 3.2361 2.2109 1.7675 1.1772 0.7566 0.5708 2.6528 0.0402 0.0678
1 0.00 0.0479 0.1359 2.2432 1.7511 1.3921 0.8076 0.7263 0.5066 1.9829 0.0211 0.0199
2 0.00 0.0753 0.1821 2.9662 2.1815 1.7794 1.0806 0.8102 0.6029 2.4676 0.0335 0.0462
3 0.00 0.0859 0.2059 3.3763 2.3638 1.8972 1.1972 0.8524 0.6463 2.7168 0.0398 0.0595
4 0.00 0.0911 0.2162 3.5579 2.4214 1.9501 1.2297 0.8595 0.6553 2.8017 0.0409 0.0632
5 0.00 0.0968 0.2264 3.7354 2.4808 2.0033 1.2756 0.8715 0.6684 2.8951 0.0417 0.0669
6 0.00 0.1007 0.2338 3.8658 2.5265 2.0437 1.3008 0.8811 0.6782 2.9610 0.0420 0.0688
8 0.00 0.1033 0.2403 4.0179 2.5598 2.0736 1.3094 0.8793 0.6770 3.0289 0.0425 0.0714
10 0.00 0.1093 0.2510 4.1866 2.6129 2.1222 1.3399 0.8849 0.6850 3.1497 0.0435 0.0764
12 0.00 0.1177 0.2637 4.3372 2.6761 2.1804 1.3851 0.8950 0.6988 3.3025 0.0450 0.0829
15 0.00 0.1280 0.2780 4.5285 2.7568 2.2491 1.4306 0.9034 0.7132 3.4909 0.0474 0.0911
1 0.35 0.0846 0.1968 3.0222 2.3925 2.0432 1.2480 0.9221 0.7101 3.0696 0.0363 0.0526
2 0.35 0.1116 0.2487 3.9266 2.7252 2.3521 1.4634 1.0040 0.7863 3.5098 0.0417 0.0669
3 0.35 0.1181 0.2609 4.1758 2.7911 2.4128 1.5041 1.0174 0.7960 3.5940 0.0420 0.0693
4 0.35 0.1270 0.2750 4.4066 2.8686 2.4863 1.5534 1.0287 0.8088 3.7416 0.0433 0.0741
5 0.35 0.1335 0.2850 4.5543 2.9165 2.5352 1.5822 1.0381 0.8184 3.8488 0.0441 0.0781
6 0.35 0.1394 0.2942 4.6847 2.9588 2.5768 1.6110 1.0462 0.8280 3.9543 0.0450 0.0823
8 0.35 0.1457 0.3047 4.8498 3.0067 2.6205 1.6362 1.0478 0.8327 4.0843 0.0455 0.0870
10 0.35 0.1514 0.3148 5.0070 3.0575 2.6634 1.6638 1.0479 0.8376 4.2225 0.0465 0.0921
12 0.35 0.1610 0.3290 5.1767 3.1314 2.7257 1.7093 1.0531 0.8524 4.4195 0.0484 0.0998
15 0.35 0.1755 0.3464 5.3610 3.2195 2.7975 1.7645 1.0597 0.8754 4.6915 0.0508 0.1104
1 0.67 0.0882 0.1948 2.9303 2.5224 2.3780 1.3513 1.0220 0.7413 3.3353 0.0254 0.0329
2 0.67 0.1274 0.2722 4.2177 2.9280 2.7639 1.6357 1.1076 0.8334 4.0553 0.0397 0.0658
3 0.67 0.1607 0.3215 4.9676 3.1920 3.0039 1.8273 1.1724 0.9175 4.6663 0.0471 0.0838
4 0.67 0.1812 0.3515 5.3837 3.3306 3.1225 1.9282 1.2006 0.9581 5.0312 0.0501 0.0925
5 0.67 0.1875 0.3606 5.5014 3.3735 3.1536 1.9489 1.2046 0.9659 5.0657 0.0501 0.0930
6 0.67 0.1927 0.3685 5.6006 3.4074 3.1854 1.9670 1.2103 0.9755 5.1527 0.0500 0.0934
8 0.67 0.2048 0.3863 5.8280 3.4913 3.2563 2.0196 1.2229 0.9997 5.4185 0.0507 0.0968
10 0.67 0.2195 0.4083 6.1146 3.5926 3.3417 2.0829 1.2379 1.0269 5.8439 0.0529 0.1042
12 0.67 0.2313 0.4253 6.3053 3.6834 3.4148 2.1299 1.2424 1.0509 6.1267 0.0541 0.1078
15 0.67 0.2502 0.4463 6.4610 3.8048 3.4981 2.1878 1.2553 1.0937 6.5567 0.0539 0.1115
'''

In [166]:
D_3_1 = '''
1 −2.25 −0.2572 −0.1490 −0.4347 −4.2937 −3.1467 −0.2036 −0.8295 1.4433 6.9993 −0.0706
2 −2.25 −0.2016 −0.1207 −0.2101 −2.3830 −2.0388 −0.0679 −0.3574 0.7673 5.0155 0.3281
3 −2.25 −0.1860 −0.1137 −0.1152 −1.8620 −1.7522 −0.0273 −0.1911 0.4939 4.5837 0.4408
4 −2.25 −0.1729 −0.1079 −0.0403 −1.4737 −1.4904 −0.0049 −0.0812 0.2810 4.2494 0.4971
5 −2.25 −0.1608 −0.1022 0.0268 −1.0895 −1.2473 0.0151 0.0086 0.1102 3.9359 0.5342
6 −2.25 −0.1477 −0.0943 0.0811 −0.7567 −0.9188 0.0536 0.1049 0.0229 3.6613 0.5956
8 −2.25 −0.1303 −0.0864 0.1561 −0.3295 −0.5557 0.0589 0.1774 −0.1969 3.1867 0.6029
10 −2.25 −0.1222 −0.0816 0.1850 −0.2947 −0.2313 0.0756 0.2124 −0.2699 3.0911 0.6147
12 −2.25 −0.1202 −0.0804 0.1778 −0.5458 −0.0606 0.0751 0.1837 −0.3057 3.1484 0.5975
15 −2.25 −0.1177 −0.0796 0.2454 −0.0949 0.1437 0.1175 0.2323 −0.3061 2.7796 0.6055
1 −1.35 −0.1755 −0.0954 −0.0548 −1.1821 −1.3782 0.1066 0.4923 0.0193 4.9652 1.1830
2 −1.35 −0.1638 −0.0917 0.0578 −0.7044 −0.9976 0.1677 0.6855 −0.1638 4.5531 1.3391
3 −1.35 −0.1476 −0.0830 0.1510 −0.1135 −0.6973 0.2391 0.8558 −0.2112 4.1693 1.4864
4 −1.35 −0.1299 −0.0724 0.2324 0.4835 −0.3488 0.3093 1.0077 −0.2037 3.7579 1.6157
5 −1.35 −0.1135 −0.0622 0.3033 1.0492 −0.0338 0.3712 1.1356 −0.1891 3.3497 1.7325
6 −1.35 −0.1018 −0.0549 0.3568 1.4382 0.2364 0.4212 1.2312 −0.1621 3.0585 1.8129
8 −1.35 −0.0869 −0.0457 0.4264 1.9392 0.5499 0.4714 1.3325 −0.1726 2.6730 1.8950
10 −1.35 −0.0764 −0.0389 0.4809 2.2867 0.7973 0.5151 1.4082 −0.1606 2.4214 1.9667
12 −1.35 −0.0725 −0.0360 0.4979 2.3248 0.9381 0.5343 1.4378 −0.1487 2.3482 1.9850
15 −1.35 −0.0995 −0.0526 0.4514 1.4710 0.7580 0.4647 1.3129 −0.2288 2.6800 1.8699
1 −0.33 −0.1012 −0.0428 0.3618 1.5276 0.5127 0.7281 1.7830 1.3193 4.0471 3.1594
2 −0.33 −0.0573 −0.0118 0.5784 3.0911 1.4869 0.9693 2.1771 1.9839 3.0584 3.5909
3 −0.33 −0.0422 −0.0011 0.6729 3.6431 1.8375 1.0526 2.3116 2.2237 2.7018 3.7242
4 −0.33 −0.0311 0.0069 0.7376 4.0358 2.0959 1.1107 2.4001 2.3569 2.4672 3.8127
5 −0.33 −0.0246 0.0113 0.7802 4.2768 2.2658 1.1452 2.4515 2.4185 2.3286 3.8674
6 −0.33 −0.0203 0.0139 0.8187 4.4673 2.4136 1.1739 2.4933 2.4632 2.2313 3.9012
8 −0.33 −0.0125 0.0196 0.8834 4.7776 2.6303 1.2268 2.5680 2.5622 2.0652 3.9734
10 −0.33 −0.0037 0.0265 0.9513 5.0791 2.8384 1.2908 2.6510 2.7083 1.9053 4.0878
12 −0.33 0.0036 0.0325 1.0015 5.3170 2.9928 1.3496 2.7201 2.8065 1.7788 4.2031
15 −0.33 0.0076 0.0366 1.0528 5.4596 3.1335 1.4054 2.7865 2.8911 1.6796 4.2446
1 0.00 −0.0865 −0.0317 0.4713 1.9700 0.8863 0.9132 1.9971 2.3719 4.2423 3.6276
2 0.00 −0.0156 0.0241 0.7608 4.1596 2.3127 1.2884 2.5714 3.6472 2.7202 4.3365
3 0.00 0.0163 0.0528 0.8858 4.9244 2.9464 1.4517 2.7829 4.2590 2.3307 4.6971
4 0.00 0.0250 0.0599 0.9514 5.2161 3.1736 1.5044 2.8537 4.4123 2.1966 4.7893
5 0.00 0.0351 0.0689 1.0122 5.4841 3.3807 1.5599 2.9273 4.5808 2.0675 4.8701
6 0.00 0.0431 0.0760 1.0579 5.7014 3.5354 1.6032 2.9815 4.7062 1.9688 4.9440
8 0.00 0.0466 0.0782 1.1068 5.8847 3.6450 1.6337 3.0117 4.7537 1.8825 5.0171
10 0.00 0.0544 0.0854 1.1713 6.0951 3.7979 1.6852 3.0740 4.8804 1.7670 5.0834
12 0.00 0.0655 0.0965 1.2399 6.3148 3.9666 1.7502 3.1567 5.0609 1.6401 5.1379
15 0.00 0.0792 0.1105 1.3255 6.5418 4.0979 1.8308 3.2460 5.2580 1.5031 5.2558
1 0.35 −0.0206 0.0260 0.7554 3.5514 2.2169 1.4107 2.6698 4.9471 3.2916 4.8725
2 0.35 0.0480 0.0869 1.0357 5.1168 3.7432 1.7230 3.0987 6.1722 2.3434 5.4402
3 0.35 0.0590 0.0951 1.1321 5.5201 4.0619 1.7888 3.1854 6.3572 2.1661 5.5705
4 0.35 0.0729 0.1079 1.2261 5.8244 4.3123 1.8616 3.2838 6.5922 2.0038 5.6799
5 0.35 0.0804 0.1147 1.2945 6.0396 4.4826 1.9109 3.3499 6.7406 1.8923 5.7411
6 0.35 0.0887 0.1228 1.3530 6.2073 4.6179 1.9551 3.4083 6.8789 1.7991 5.7903
8 0.35 0.0966 0.1301 1.4351 6.3806 4.7510 2.0020 3.4657 6.9907 1.6894 5.8392
10 0.35 0.1079 0.1414 1.5136 6.5395 4.8577 2.0538 3.5260 7.1369 1.5868 5.8987
12 0.35 0.1236 0.1583 1.6162 6.6754 4.9870 2.1245 3.6152 7.3582 1.4726 5.9758
15 0.35 0.1453 0.1818 1.7480 6.7967 5.0717 2.2113 3.7237 7.6300 1.3410 6.0494
1 0.67 −0.0266 0.0137 0.8367 4.0135 2.6819 1.5712 2.8040 5.8748 3.2311 5.3218
2 0.67 0.0415 0.0745 1.1989 5.6474 4.7547 1.9307 3.3204 7.4331 2.3114 6.0213
3 0.67 0.0934 0.1279 1.4867 6.2412 5.3512 2.1600 3.6434 8.5012 1.8861 6.3891
4 0.67 0.1280 0.1647 1.6699 6.5229 5.6519 2.2936 3.8247 9.0634 1.6603 6.5475
5 0.67 0.1381 0.1753 1.6956 6.5953 5.7599 2.3313 3.8791 9.1988 1.5873 6.5906
6 0.67 0.1479 0.1862 1.7387 6.6631 5.8026 2.3613 3.9197 9.3191 1.5328 6.6206
8 0.67 0.1705 0.2108 1.8712 6.7773 5.8982 2.4340 4.0154 9.5984 1.4101 6.6888
10 0.67 0.2007 0.2442 2.0989 6.8671 6.0220 2.5265 4.1390 9.9424 1.2597 6.7736
12 0.67 0.2343 0.2815 2.2409 6.8157 6.0524 2.5889 4.2329 10.2176 1.1739 6.8343
15 0.67 0.3005 0.3541 2.3958 6.6693 6.0528 2.6570 4.3552 10.6631 1.0736 6.7895
'''

In [167]:
D_3_2 = '''
1 −2.25 0.0162 −0.0026 0.2327 0.0603 0.1690 −0.1685 −0.0496 −0.4578 0.8483 0.0136 −0.0056
2 −2.25 0.0071 0.0105 0.4252 0.2007 0.2915 −0.0511 −0.0063 −0.3577 0.7711 0.0127 −0.0044
3 −2.25 0.0052 0.0137 0.4437 0.2940 0.3362 −0.0062 0.0082 −0.3268 0.7897 0.0120 −0.0046
4 −2.25 0.0039 0.0184 0.5414 0.3583 0.3519 0.0215 0.0104 −0.3150 0.8128 0.0115 −0.0049
5 −2.25 0.0034 0.0226 0.6275 0.4151 0.3664 0.0422 0.0121 −0.3061 0.8425 0.0111 −0.0052
6 −2.25 0.0024 0.0253 0.6532 0.4865 0.4103 0.0809 0.0211 −0.2796 0.8506 0.0109 −0.0053
8 −2.25 0.0043 0.0352 0.9447 0.4977 0.4040 0.0862 0.0152 −0.2907 0.9367 0.0103 −0.0059
10 −2.25 0.0061 0.0376 0.9511 0.5449 0.4371 0.1107 0.0198 −0.2760 0.9926 0.0099 −0.0063
12 −2.25 0.0091 0.0390 0.9693 0.5704 0.4617 0.1286 0.0259 −0.2685 1.0504 0.0095 −0.0069
15 −2.25 0.0139 0.0461 1.0573 0.5760 0.5067 0.1428 0.0298 −0.2673 1.1424 0.0092 −0.0072
1 −1.35 0.0156 0.0367 0.8631 0.1938 0.2938 0.0968 0.2437 −0.0279 0.8106 0.0135 −0.0003
2 −1.35 0.0148 0.0442 0.9984 0.2811 0.3605 0.1476 0.2523 0.0055 0.8477 0.0135 0.0006
3 −1.35 0.0150 0.0505 1.1153 0.3956 0.4281 0.1919 0.2652 0.0265 0.8973 0.0135 0.0013
4 −1.35 0.0163 0.0571 1.2360 0.5107 0.4869 0.2389 0.2781 0.0466 0.9557 0.0137 0.0020
5 −1.35 0.0181 0.0631 1.3417 0.6075 0.5331 0.2789 0.2921 0.0616 1.0029 0.0140 0.0028
6 −1.35 0.0200 0.0680 1.4100 0.6876 0.5806 0.3158 0.3023 0.0804 1.0534 0.0141 0.0034
8 −1.35 0.0233 0.0758 1.5460 0.7537 0.6208 0.3431 0.3113 0.0861 1.1348 0.0144 0.0040
10 −1.35 0.0267 0.0818 1.6243 0.8118 0.6597 0.3711 0.3220 0.0965 1.2072 0.0148 0.0048
12 −1.35 0.0287 0.0861 1.6114 0.8463 0.6918 0.3916 0.3232 0.1070 1.2760 0.0151 0.0053
15 −1.35 0.0316 0.0838 1.5436 0.7758 0.6729 0.3823 0.3143 0.0959 1.3534 0.0152 0.0052
1 −0.33 0.0454 0.1167 2.0352 1.2869 0.9182 0.6066 0.6230 0.3723 1.5622 0.0218 0.0199
2 −0.33 0.0573 0.1427 2.5107 1.5421 1.1428 0.7554 0.6532 0.4241 1.8363 0.0290 0.0348
3 −0.33 0.0632 0.1539 2.7013 1.6367 1.2232 0.7945 0.6540 0.4375 1.9657 0.0326 0.0418
4 −0.33 0.0674 0.1613 2.8220 1.7037 1.2712 0.8271 0.6589 0.4462 2.0492 0.0337 0.0452
5 −0.33 0.0698 0.1659 2.9050 1.7463 1.2990 0.8470 0.6605 0.4503 2.1075 0.0342 0.0469
6 −0.33 0.0726 0.1707 2.9776 1.7803 1.3294 0.8725 0.6605 0.4541 2.1676 0.0349 0.0492
8 −0.33 0.0780 0.1810 3.1548 1.8363 1.3747 0.8951 0.6638 0.4601 2.2838 0.0361 0.0532
10 −0.33 0.0843 0.1924 3.3411 1.9040 1.4254 0.9240 0.6715 0.4717 2.4083 0.0382 0.0580
12 −0.33 0.0903 0.2018 3.4870 1.9633 1.4732 0.9624 0.6823 0.4831 2.5165 0.0395 0.0620
15 −0.33 0.1004 0.2125 3.5937 2.0114 1.5152 0.9953 0.6954 0.4929 2.6640 0.0407 0.0667
1 0.00 0.0553 0.1422 2.5052 1.5839 1.1905 0.6610 0.6707 0.4255 1.9794 0.0211 0.0191
2 0.00 0.0854 0.1905 3.3042 1.9892 1.5347 0.9051 0.7468 0.5171 2.4654 0.0339 0.0450
3 0.00 0.0977 0.2158 3.7720 2.1573 1.6376 1.0118 0.7870 0.5583 2.7233 0.0405 0.0582
4 0.00 0.1034 0.2264 3.9680 2.2115 1.6834 1.0410 0.7927 0.5671 2.8073 0.0416 0.0618
5 0.00 0.1098 0.2371 4.1618 2.2665 1.7289 1.0818 0.8033 0.5795 2.9011 0.0424 0.0655
6 0.00 0.1141 0.2448 4.3038 2.3083 1.7634 1.1037 0.8117 0.5892 2.9671 0.0427 0.0675
8 0.00 0.1169 0.2514 4.4633 2.3379 1.7893 1.1084 0.8087 0.5880 3.0333 0.0431 0.0700
10 0.00 0.1234 0.2625 4.6457 2.3851 1.8303 1.1327 0.8130 0.5959 3.1548 0.0442 0.0750
12 0.00 0.1327 0.2758 4.8132 2.4408 1.8785 1.1703 0.8219 0.6089 3.3109 0.0457 0.0815
15 0.00 0.1452 0.2908 5.0290 2.5112 1.9346 1.2077 0.8289 0.6225 3.5040 0.0481 0.0898
1 0.35 0.0966 0.2061 3.3811 2.1820 1.7634 1.0457 0.8506 0.6144 3.0798 0.0368 0.0513
2 0.35 0.1261 0.2604 4.3837 2.4911 2.0326 1.2374 0.9261 0.6859 3.5233 0.0424 0.0654
3 0.35 0.1336 0.2730 4.6467 2.5533 2.0868 1.2715 0.9368 0.6953 3.6030 0.0427 0.0678
4 0.35 0.1435 0.2877 4.8982 2.6238 2.1496 1.3137 0.9465 0.7075 3.7509 0.0440 0.0726
5 0.35 0.1507 0.2980 5.0574 2.6665 2.1913 1.3375 0.9545 0.7166 3.8583 0.0448 0.0766
6 0.35 0.1572 0.3076 5.1993 2.7033 2.2262 1.3612 0.9614 0.7257 3.9650 0.0457 0.0808
8 0.35 0.1641 0.3185 5.3738 2.7455 2.2630 1.3797 0.9614 0.7303 4.0942 0.0462 0.0854
10 0.35 0.1702 0.3290 5.5444 2.7892 2.2978 1.4010 0.9602 0.7353 4.2339 0.0472 0.0906
12 0.35 0.1809 0.3438 5.7342 2.8539 2.3490 1.4384 0.9645 0.7491 4.4336 0.0491 0.0982
15 0.35 0.1989 0.3622 5.9449 2.9298 2.4061 1.4841 0.9698 0.7708 4.7116 0.0515 0.1090
1 0.67 0.1002 0.2028 3.2325 2.3144 2.0747 1.1252 0.9344 0.6444 3.3241 0.0253 0.0318
2 0.67 0.1437 0.2833 4.6291 2.6909 2.4091 1.3718 1.0115 0.7296 4.0421 0.0402 0.0642
3 0.67 0.1809 0.3351 5.4728 2.9272 2.6053 1.5376 1.0721 0.8074 4.6662 0.0478 0.0819
4 0.67 0.2036 0.3668 5.9472 3.0476 2.7016 1.6237 1.0986 0.8451 5.0417 0.0510 0.0906
5 0.67 0.2106 0.3763 6.0740 3.0840 2.7281 1.6377 1.1013 0.8525 5.0756 0.0509 0.0911
6 0.67 0.2161 0.3846 6.1853 3.1106 2.7532 1.6507 1.1061 0.8612 5.1603 0.0508 0.0915
8 0.67 0.2294 0.4033 6.4405 3.1823 2.8091 1.6939 1.1165 0.8842 5.4287 0.0515 0.0949
10 0.67 0.2455 0.4264 6.7621 3.2690 2.8766 1.7459 1.1289 0.9096 5.8682 0.0537 0.1023
12 0.67 0.2589 0.4444 6.9823 3.3476 2.9348 1.7854 1.1326 0.9318 6.1552 0.0550 0.1060
15 0.67 0.2827 0.4667 7.1696 3.4507 2.9976 1.8342 1.1434 0.9720 6.5935 0.0548 0.1098
'''

In [168]:
# process_and_merge_data(header1, D_1_1, header2, D_1_2, './../TMB03/TMB03_AOFe00.csv')
# process_and_merge_data(header1, D_2_1, header2, D_2_2, './../TMB03/TMB03_AOFe03.csv')
# process_and_merge_data(header1, D_3_1, header2, D_3_2, './../TMB03/TMB03_AOFe05.csv')

In [169]:
D_1= pd.read_csv('./../TMB03/TMB03_AOFe00.csv')
D_2= pd.read_csv('./../TMB03/TMB03_AOFe03.csv')
D_3= pd.read_csv('./../TMB03/TMB03_AOFe05.csv')

In [170]:
D_1['AoFe'] = 0.0
D_2['AoFe'] = 0.3
D_3['AoFe'] = 0.5

In [171]:
D_all = TB_reindex(pd.concat([D_1,D_2,D_3]))

In [172]:
D_all

,Age,ZoH,CN1,CN2,Ca4227,G4300,Fe4383,Ca4455,Fe4531,C24668,...,Mgb,Fe5270,Fe5335,Fe5406,Fe5709,Fe5782,NaD,TiO1,TiO2,AoFe
0,1,-2.25,-0.2632,-0.1536,-0.4260,-4.3872,-2.8764,-0.1924,-0.8226,1.6082,...,0.1794,0.1715,0.2673,-0.1296,-0.0573,-0.4248,0.8827,0.0140,-0.0048,0.0
1,2,-2.25,-0.2033,-0.1214,-0.2038,-2.4775,-1.8406,-0.0604,-0.3240,0.9274,...,0.3366,0.3408,0.4326,0.0056,-0.0074,-0.3342,0.8046,0.0132,-0.0037,0.0
2,3,-2.25,-0.1871,-0.1137,-0.1132,-1.9934,-1.4889,-0.0243,-0.1584,0.6396,...,0.3632,0.4333,0.4781,0.0519,0.0094,-0.3070,0.8133,0.0125,-0.0039,0.0
3,4,-2.25,-0.1740,-0.1077,-0.0440,-1.6085,-1.2146,-0.0036,-0.0539,0.4111,...,0.4429,0.4910,0.5023,0.0725,0.0119,-0.2974,0.8275,0.0121,-0.0042,0.0
4,5,-2.25,-0.1621,-0.1021,0.0173,-1.2265,-0.9724,0.0149,0.0312,0.2273,...,0.5125,0.5421,0.5246,0.0835,0.0140,-0.2900,0.8497,0.0117,-0.0045,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,6,0.67,0.1479,0.1862,1.7387,6.6631,5.8026,2.3613,3.9197,9.3191,...,6.1853,3.1106,2.7532,1.6507,1.1061,0.8612,5.1603,0.0508,0.0915,0.5
176,8,0.67,0.1705,0.2108,1.8712,6.7773,5.8982,2.4340,4.0154,9.5984,...,6.4405,3.1823,2.8091,1.6939,1.1165,0.8842,5.4287,0.0515,0.0949,0.5
177,10,0.67,0.2007,0.2442,2.0989,6.8671,6.0220,2.5265,4.1390,9.9424,...,6.7621,3.2690,2.8766,1.7459,1.1289,0.9096,5.8682,0.0537,0.1023,0.5
178,12,0.67,0.2343,0.2815,2.2409,6.8157,6.0524,2.5889,4.2329,10.2176,...,6.9823,3.3476,2.9348,1.7854,1.1326,0.9318,6.1552,0.0550,0.1060,0.5


In [174]:
D_all.to_csv('./../TMB03/TMB03.csv')